# Trabajo práctico integrador - Visión por Computadoras II
## Carrera de Especialización en Inteligencia Artificial - Cohorte 17
### Autores:
* Piñero, Juan Cruz 
* Lloveras, Alejandro
* Méndez, Diego Martín

**Objetivo del trabajo**

Utilizar modelos de *Computer Vision* para clasificar enfermedades de plantas (38 clases).

In [1]:
# Importamos librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Dataset

Dataset de Kaggle con ~87 mil imágenes RGB de hojas de cultivos sanas y enfermas, categorizadas en 38 clases diferentes.

**Link:** https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdallahalidev/plantvillage-dataset")

print("Path to dataset files:", path)

/opt/anaconda3/envs/cv-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.04G/2.04G [01:48<00:00, 20.1MB/s]

Extracting model files...


Path to dataset files: /Users/alejandrolloveras/.cache/kagglehub/datasets/abdallahalidev/plantvillage-dataset/versions/3


## Importación de imágenes

In [3]:
import os
import logging

In [4]:
# CONFIGURACIÓN INICIAL:

# Ruta de acceso al dataset
ROOT_DIR = '/Users/alejandrolloveras/.cache/kagglehub/datasets/abdallahalidev/plantvillage-dataset/versions/3/plantvillage dataset/color'

# Separador usado en los nombres de las subcarpetas
SEPARATOR = '___' # Separa entre 'group' y 'class'

# Extensiones de archivo de imagen a considerar (en minúsculas)
IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp'}

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [5]:
def process_image_directory(root_dir, separator='___', valid_extensions=None):
    """
    Recorre un directorio raíz, extrae rutas de imágenes y metadatos de subcarpetas.

    Args:
        root_dir (str): La ruta al directorio raíz que contiene las subcarpetas de clases.
        separator (str): El separador utilizado en los nombres de las subcarpetas
                         para dividir 'group' y 'class'.
        valid_extensions (set, optional): Un conjunto de extensiones de archivo (en minúsculas,
                                          incluyendo el punto) a considerar como imágenes.
                                          Si es None, se usará un conjunto predeterminado.

    Returns:
        pandas.DataFrame: Un DataFrame con las columnas 'image_path', 'filename', 'group', 'class'.
                          Retorna un DataFrame vacío si el directorio raíz no existe o
                          no se encuentran imágenes válidas.
    """
    if valid_extensions is None:
        valid_extensions = IMAGE_EXTENSIONS # Verifica que sean archivos de imagen

    data = []
    output_columns = ['image_path', 'filename', 'group', 'class'] # Define la estructura para el Dataframe

    if not os.path.isdir(root_dir):
        logging.error(f"El directorio raíz especificado no existe: {root_dir}")
        return pd.DataFrame(data, columns=output_columns) # Asegura que exista el directorio

    logging.info(f"Comenzando el procesamiento del directorio: {root_dir}")

    # Exploración de cada sucarpeta
    for entry in os.scandir(root_dir):
        if entry.is_dir():
            subdir_name = entry.name
            subdir_path = entry.path
            logging.info(f"Procesando subdirectorio: {subdir_name}")

            if separator in subdir_name:
                try:
                    group_name, class_name = subdir_name.split(separator, 1)
                except ValueError:
                    logging.warning(f"No se pudo dividir el nombre '{subdir_name}' usando '{separator}'. Saltando directorio.")
                    continue

                for sub_entry in os.scandir(subdir_path):
                    if sub_entry.is_file():
                        file_name = sub_entry.name
                        file_path = sub_entry.path

                        _, extension = os.path.splitext(file_name)
                        if extension.lower() in valid_extensions:
                            relative_path = os.path.relpath(os.path.dirname(file_path), root_dir) # Carpeta dónde encontrar a imagen
                            
                            # Almacena toda la información
                            data.append({
                                'image_path': f"{relative_path}/",
                                'filename': file_name,
                                'group': group_name,
                                'class': class_name
                            })
                        else:
                            logging.debug(f"Archivo omitido (no es imagen válida): {file_path}")
            else:
                logging.warning(f"El nombre del subdirectorio '{subdir_name}' no contiene el separador '{separator}'. Saltando.")

    if not data:
        logging.warning("No se encontraron imágenes válidas en la estructura de directorios especificada.")

    # Construye el DataFrame
    df = pd.DataFrame(data, columns=output_columns)
    logging.info(f"Proceso completado. Se encontraron {len(df)} imágenes.")
    return df

In [6]:
df = process_image_directory(ROOT_DIR)

2025-03-31 13:30:15,389 - INFO - Comenzando el procesamiento del directorio: /Users/alejandrolloveras/.cache/kagglehub/datasets/abdallahalidev/plantvillage-dataset/versions/3/plantvillage dataset/color
2025-03-31 13:30:15,397 - INFO - Procesando subdirectorio: Strawberry___healthy
2025-03-31 13:30:15,415 - INFO - Procesando subdirectorio: Grape___Black_rot
2025-03-31 13:30:15,446 - INFO - Procesando subdirectorio: Potato___Early_blight
2025-03-31 13:30:15,473 - INFO - Procesando subdirectorio: Blueberry___healthy
2025-03-31 13:30:15,512 - INFO - Procesando subdirectorio: Corn_(maize)___healthy
2025-03-31 13:30:15,536 - INFO - Procesando subdirectorio: Tomato___Target_Spot
2025-03-31 13:30:15,565 - INFO - Procesando subdirectorio: Peach___healthy
2025-03-31 13:30:15,573 - INFO - Procesando subdirectorio: Potato___Late_blight
2025-03-31 13:30:15,592 - INFO - Procesando subdirectorio: Tomato___Late_blight
2025-03-31 13:30:15,632 - INFO - Procesando subdirectorio: Tomato___Tomato_mosaic_vi

In [7]:
df.sample(5)

,image_path,filename,group,class
48895,Strawberry___Leaf_scorch/,1f7f3e65-fb9f-4a05-8304-26850067d617___RS_L.Sc...,Strawberry,Leaf_scorch
23140,Grape___healthy/,7dcbe0fa-4dc1-4234-97d4-9c19391fbe21___Mt.N.V_...,Grape,healthy
22670,Tomato___Bacterial_spot/,d1c21ca6-efc2-48f3-a1be-1593320db75c___GCREC_B...,Tomato,Bacterial_spot
17190,Orange___Haunglongbing_(Citrus_greening)/,0df51ff0-4618-4eac-ba38-04b7a781294c___CREC_HL...,Orange,Haunglongbing_(Citrus_greening)
34587,Tomato___Tomato_Yellow_Leaf_Curl_Virus/,2ff9d604-2e15-4baa-974d-2038df4501e1___UF.GRC_...,Tomato,Tomato_Yellow_Leaf_Curl_Virus


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54305 entries, 0 to 54304
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_path  54305 non-null  object
 1   filename    54305 non-null  object
 2   group       54305 non-null  object
 3   class       54305 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [9]:
df.describe()

,image_path,filename,group,class
count,54305,54305,54305,54305
unique,38,54305,14,21
top,Orange___Haunglongbing_(Citrus_greening)/,c7f4b7b2-b13c-476e-a936-c91ce39749a3___RS_HL 6...,Tomato,healthy
freq,5507,1,18160,15084


In [10]:
print("Conteo por grupo:")
print(df['group'].value_counts())

Conteo por grupo:
group
Tomato                     18160
Orange                      5507
Soybean                     5090
Grape                       4062
Corn_(maize)                3852
Apple                       3171
Peach                       2657
Pepper,_bell                2475
Potato                      2152
Cherry_(including_sour)     1906
Squash                      1835
Strawberry                  1565
Blueberry                   1502
Raspberry                    371
Name: count, dtype: int64


In [11]:
print("Conteo por clase:")
print(df['class'].value_counts())

Conteo por clase:
class
healthy                                15084
Haunglongbing_(Citrus_greening)         5507
Bacterial_spot                          5421
Tomato_Yellow_Leaf_Curl_Virus           5357
Late_blight                             2909
                                       ...  
Leaf_Mold                                952
Apple_scab                               630
Cercospora_leaf_spot Gray_leaf_spot      513
Tomato_mosaic_virus                      373
Cedar_apple_rust                         275
Name: count, Length: 21, dtype: int64
